In [ ]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load the stock price data from a CSV file
data = pd.read_csv("./NIFTY50/ADANI_ENTERPRISES.csv")

# Define feature columns (X) and target column (Y)
X = data[["High", "Low", "Close", "Adj Close", "Volume"]]
Y = data["Open"]

# Convert the data to numpy arrays for handling NaN values
X_raw = np.array(X)
Y_raw = np.array(Y)

# Handle NaN values by replacing them with 0
X_raw[np.isnan(X_raw)] = 0
Y_raw[np.isnan(Y_raw)] = 0

# Normalize the feature data (standardization: mean = 0, std = 1)
X = (X_raw - X_raw.mean(axis=0)) / X_raw.std(axis=0)
Y = (Y_raw - Y_raw.mean()) / Y_raw.std()

# Split the data into training and testing sets (80% training, 20% testing)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Build the neural network model
model = Sequential([
    Input(shape=(X_train.shape[1],)),  # Input layer
    Dense(64, activation='relu'),      # First hidden layer with ReLU activation
    Dense(32, activation='relu'),      # Second hidden layer with ReLU activation
    Dense(1)                           # Output layer with 1 neuron (predicting a single value)
])

# Compile the model with Adam optimizer and mean squared error loss function
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model on the training data
model.fit(X_train, Y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model on the testing data
loss = model.evaluate(X_test, Y_test)
print(f'Test loss: {loss}')

# Make predictions on the testing data
predictions = model.predict(X_train)


# Plot the actual vs. predicted stock prices
plt.figure(figsize=(14, 7))
plt.plot(np.arange(len(Y)), Y, label='Actual')  # Plot actual stock prices
plt.plot(np.arange(len(predictions)), predictions, label='Predicted')  # Plot predicted stock prices
plt.title('Actual vs Predicted Stock Prices')
plt.xlabel('Time')
plt.ylabel('Normalized Price')
plt.legend()

# Show the plot
plt.show()


Epoch 1/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.2285 - val_loss: 9.3088e-04
Epoch 2/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0017 - val_loss: 4.5082e-04
Epoch 3/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0015 - val_loss: 4.4766e-04
Epoch 4/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 8.1723e-04 - val_loss: 4.7817e-04
Epoch 5/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 8.7624e-04 - val_loss: 3.3576e-04
Epoch 6/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 5.1486e-04 - val_loss: 5.5200e-04
Epoch 7/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 6.5193e-04 - val_loss: 7.1304e-04
Epoch 8/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 8.1953e-04 - val_loss: 7.1718e-04
Epoch 9/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0011 - val_loss: 3.7462e-04
Epoch 10/10
 56/105 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4.2054e-04

In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM

# Load data
data = pd.read_csv('./NIFTY50/SBI_BANK.csv')
data = data['Close'][-500:].values.reshape(-1, 1)

# Normalize data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# Create training and testing datasets
train_size = int(len(scaled_data) * 0.8)
train_data = scaled_data[:train_size]
test_data = scaled_data[train_size:]

# Create dataset with lookback
def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset) - look_back - 1):
        a = dataset[i:(i + look_back), 0]
        X.append(a)
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)

look_back = 60
X_train, y_train = create_dataset(train_data, look_back)
X_test, y_test = create_dataset(test_data, look_back)

# Reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Build LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(look_back, 1)))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

# Compile and train the model
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, batch_size=1, epochs=1)

# Make predictions
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

# Inverse transform predictions
train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
y_train = scaler.inverse_transform([y_train])
y_test = scaler.inverse_transform([y_test])

# Plot results
plt.figure(figsize=(16, 8))
plt.plot(data, label='Actual Stock Price')
plt.plot(range(look_back, look_back + len(train_predict)), train_predict, label='Training Prediction')
plt.plot(range(look_back + len(train_predict) + look_back + 1, len(data) - 1), test_predict, label='Testing Prediction')
plt.legend()
plt.show()



339/339 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0841
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 


ValueError: Found array with dim 3. None expected <= 2.